In [1]:
spark

## Processing `Comments.xml`

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, LongType
from datetime import datetime

In [4]:
# Dataset path 
dataset_bucket = 's3://stackoverflow-dataset-2023/dataset/raw'
dataset_comments = f"{dataset_bucket}/Users.xml"

In [6]:
rdd = spark.sparkContext.textFile(dataset_comments)

rdd.count()

19307024

In [38]:
def row_parser(row):
    row_len = len(row.split('"')) 
    result = [None] * 12
    if row_len == 19:
        result = (int(row.split('"')[1]), 
                     int(row.split('"')[3]),
                     datetime.strptime(row.split('"')[5], "%Y-%m-%dT%H:%M:%S.%f"), 
                     row.split('"')[7], 
                     datetime.strptime(row.split('"')[9], "%Y-%m-%dT%H:%M:%S.%f"), 
                     row.split('"')[11], 
                     row.split('"')[13], 
                     row.split('"')[15],  
                     row.split('"')[17],
                     None, 
                     None, 
                     None)
                    
    elif row_len == 25:
        result = (int(row.split('"')[1]), 
                     int(row.split('"')[3]),  
                     datetime.strptime(row.split('"')[5], "%Y-%m-%dT%H:%M:%S.%f"), 
                     row.split('"')[7], 
                     datetime.strptime(row.split('"')[9], "%Y-%m-%dT%H:%M:%S.%f"), 
                     row.split('"')[11], 
                     row.split('"')[13], 
                     row.split('"')[15],  
                     row.split('"')[17],
                     row.split('"')[19], 
                     row.split('"')[21],  
                     row.split('"')[23],                  
                 )
        
    return result

In [ ]:
parsed_rdd = rdd.map(lambda row: row.strip()) \
   .filter(lambda row: row.startswith("<row")) \
   .map(lambda row: row[4:-3]) \
   .map(lambda row: row.strip()) \
   .map(row_parser)

In [66]:
parsed_rdd = rdd.map(lambda row: row.strip()) \
   .filter(lambda row: row.startswith("<row")) \
   .map(lambda row: row[4:-3]) \
   .map(lambda row: row.strip()) \
   .filter(lambda row: len(row.split('"')) > 26 ) 

In [71]:
a = range(10, 27)
list(a)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

In [ ]:
for i in a:
    parsed_rdd = rdd.map(lambda row: row.strip()) \
       .filter(lambda row: row.startswith("<row")) \
       .map(lambda row: row[4:-3]) \
       .map(lambda row: row.strip()) \
       .filter(lambda row: len(row.split('"')) > i ) 
    
    print(i, parsed_rdd.count())

10 19307021


11 19307021


12 19307021


13 19307021


14 19307021


...

17 19306967


18 19306967


19 16916392


20 16916392


21 6864191


In [67]:
parsed_rdd.count()

1342291

In [41]:
parsed_rdd.take(100)

[('-1011',
  '1',
  datetime.datetime(2022, 11, 9, 20, 49, 3, 637000),
  'AWS',
  datetime.datetime(2022, 11, 9, 20, 49, 3, 637000),
  '&lt;p&gt;Amazon Web Services (AWS) is the world’s most comprehensive and broadly adopted cloud platform, offering over 200 fully featured services from data centers globally. The AWS Collective is a community-driven site with resources for AWS developers.&lt;/p&gt;&#xA;',
  '0',
  '0',
  '0',
  None,
  None,
  None),
 ('-1010',
  '1',
  datetime.datetime(2022, 10, 25, 19, 18, 31, 537000),
  'Microsoft Azure',
  datetime.datetime(2022, 10, 25, 19, 18, 31, 537000),
  '&lt;p&gt;Azure Collective&lt;/p&gt;&#xA;',
  '0',
  '0',
  '0',
  None,
  None,
  None),
 ('-1009',
  '1',
  datetime.datetime(2022, 5, 17, 15, 6, 54, 890000),
  'WSO2',
  datetime.datetime(2022, 5, 17, 15, 6, 54, 890000),
  '&lt;p&gt;WSO2 solutions give enterprises the flexibility to deploy applications and services on-premises, on private or public clouds, or in hybrid environments. Our c

In [57]:
a = 'Id="2" Reputation="6046" CreationDate="2008-07-31T14:22:31.287" DisplayName="Geoff Dalgas" LastAccessDate="2022-11-30T22:05:42.253" WebsiteUrl="http://stackoverflow.com" Location="Corvallis, OR" AboutMe="&lt;p&gt;Dev #2 who helped create Stack Overflow currently working at Microsoft. I love all people willing to share their knowledge. Without community we are nothing.&lt;/p&gt;&#xA;&lt;p&gt;Find me on:&lt;/p&gt;&#xA;&lt;p&gt;&lt;a href=&quot;http://www.twitter.com/SuperDalgas&quot; rel=&quot;nofollow noreferrer&quot;&gt;Twitter&lt;/a&gt;&#xA;&lt;br&gt;&lt;br&gt;&#xA;&lt;a href=&quot;http://blog.stackoverflow.com/2009/05/welcome-stack-overflow-valued-associate-00003/&quot;&gt;Stack Overflow Valued Associate #00003&lt;/a&gt;&lt;/p&gt;&#xA;" Views="31564" UpVotes="671" DownVotes="88" ProfileImageUrl="https://i.stack.imgur.com/nDllk.png?s=128&amp;g=1" AccountId="2"'
for i, j in enumerate(a.split('"')):
    print(i, j)

0 Id=
1 2
2  Reputation=
3 6046
4  CreationDate=
5 2008-07-31T14:22:31.287
6  DisplayName=
7 Geoff Dalgas
8  LastAccessDate=
9 2022-11-30T22:05:42.253
10  WebsiteUrl=
11 http://stackoverflow.com
12  Location=
13 Corvallis, OR
14  AboutMe=
15 &lt;p&gt;Dev #2 who helped create Stack Overflow currently working at Microsoft. I love all people willing to share their knowledge. Without community we are nothing.&lt;/p&gt;&#xA;&lt;p&gt;Find me on:&lt;/p&gt;&#xA;&lt;p&gt;&lt;a href=&quot;http://www.twitter.com/SuperDalgas&quot; rel=&quot;nofollow noreferrer&quot;&gt;Twitter&lt;/a&gt;&#xA;&lt;br&gt;&lt;br&gt;&#xA;&lt;a href=&quot;http://blog.stackoverflow.com/2009/05/welcome-stack-overflow-valued-associate-00003/&quot;&gt;Stack Overflow Valued Associate #00003&lt;/a&gt;&lt;/p&gt;&#xA;
16  Views=
17 31564
18  UpVotes=
19 671
20  DownVotes=
21 88
22  ProfileImageUrl=
23 https://i.stack.imgur.com/nDllk.png?s=128&amp;g=1
24  AccountId=
25 2
26 


In [19]:
b = 'Id="-1" Reputation="1" CreationDate="2008-07-31T00:00:00.000" DisplayName="Community" LastAccessDate="2008-08-26T00:16:53.810" WebsiteUrl="https://meta.stackexchange.com" Location="on the server farm" AboutMe="&lt;p&gt;Hi, I\'m not really a person.&lt;/p&gt;&#xA;&lt;p&gt;I\'m a background process that helps keep this site clean!&lt;/p&gt;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&lt;ul&gt;&#xA;  &lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;  &lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;  &lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;  &lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;  &lt;li&gt;&lt;a href=&quot;https://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" Views="115570" UpVotes="575542" DownVotes="1468464" AccountId="-1"'
for i, j in enumerate(b.split('"')):
    print(i, j)

0 Id=
1 -1
2  Reputation=
3 1
4  CreationDate=
5 2008-07-31T00:00:00.000
6  DisplayName=
7 Community
8  LastAccessDate=
9 2008-08-26T00:16:53.810
10  WebsiteUrl=
11 https://meta.stackexchange.com
12  Location=
13 on the server farm
14  AboutMe=
15 &lt;p&gt;Hi, I'm not really a person.&lt;/p&gt;&#xA;&lt;p&gt;I'm a background process that helps keep this site clean!&lt;/p&gt;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&lt;ul&gt;&#xA;  &lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;  &lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;  &lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;  &lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;  &lt;li&gt;&lt;a href=&quot;https://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;
16  Views=
17 115570
18  UpVotes=
19 575542
20

In [8]:
a = 'Id="-1011" Reputation="1" CreationDate="2022-11-09T20:49:03.637" DisplayName="AWS" LastAccessDate="2022-11-09T20:49:03.637" AboutMe="&lt;p&gt;Amazon Web Services (AWS) is the world’s most comprehensive and broadly adopted cloud platform, offering over 200 fully featured services from data centers globally. The AWS Collective is a community-driven site with resources for AWS developers.&lt;/p&gt;&#xA;" Views="0" UpVotes="0" DownVotes="0"'
for i, j in enumerate(a.split('"')):
    print(i, j)

0 Id=
1 -1011
2  Reputation=
3 1
4  CreationDate=
5 2022-11-09T20:49:03.637
6  DisplayName=
7 AWS
8  LastAccessDate=
9 2022-11-09T20:49:03.637
10  AboutMe=
11 &lt;p&gt;Amazon Web Services (AWS) is the world’s most comprehensive and broadly adopted cloud platform, offering over 200 fully featured services from data centers globally. The AWS Collective is a community-driven site with resources for AWS developers.&lt;/p&gt;&#xA;
12  Views=
13 0
14  UpVotes=
15 0
16  DownVotes=
17 0
18 


In [ ]:
   .filter(lambda row: row.startswith("<row")) \
   .map(lambda row: row[4:-3]) \
   .map(lambda row: row.strip()) \
   .map(lambda row: (int(row.split('"')[1]), row.split('"')[3], int(row.split('"')[5]), int(row.split('"')[7]), int(row.split('"')[9])) if len(row) == 11 else (int(row.split('"')[1]), row.split('"')[3], int(row.split('"')[5]), None, None)) 
   